In [1]:
!pip install pytorch-lightning
!pip install npy-append-array

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 15.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pytorch_lightning as p1
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

from google.colab import drive

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DRIVE = 'drive/MyDrive/CS 8395 - C-DiffNet/MNIST_Epochs/'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:00<00:00, 11563172.71it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 340979.50it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 3153904.57it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 8239848.08it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
class MLP(p1.LightningModule):
  def __init__(self):
    super().__init__()

    self.save_hyperparameters()

    self.l1 = nn.Linear(28*28, 100)
    self.l2 = nn.Linear(100, 50)
    self.l3 = nn.Linear(50, 32)
    self.classifier = nn.Linear(32, 10)
    self.activation = nn.ReLU()

    self.loss_func = nn.CrossEntropyLoss()

  def forward(self, x):
    x = x.view(-1, 28*28)
    x = self.activation(self.l1(x))
    x = self.activation(self.l2(x))
    x = self.activation(self.l3(x))
    out = self.classifier(x)
    return out

  def configure_optimizers(self):
    # Create an optimizer object
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer


  def training_step(self, batch, batch_idx):
    x, label = batch
    x = x.to(device)
    y = x.to(device)
    logits = self.forward(x)
    loss = self.loss_func(logits, torch.tensor([label]))
    self.log('train_loss', loss)
    return loss

  def on_train_epoch_end(self):
    filename = f"checkpoints/checkpoint_epoch_{self.current_epoch}_batch_{self.global_step}.ckpt"
    checkpoint_path = os.path.join(self.trainer.logger.log_dir, filename)
    self.trainer.save_checkpoint(checkpoint_path)
    return

In [13]:
model = MLP()

# Get the best params for the other layers first (max to 20, so go to 20 epochs)
main_optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.CrossEntropyLoss()

for epoch in range(20):
  for images, labels in train_loader:
    main_optimizer.zero_grad()
    output = model(images.view(-1, 28*28))
    loss = loss_func(output, labels)
    loss.backward()
    main_optimizer.step()
  print(f"Completed training epoch {epoch}")

Completed training epoch 0
Completed training epoch 1
Completed training epoch 2
Completed training epoch 3
Completed training epoch 4
Completed training epoch 5
Completed training epoch 6
Completed training epoch 7
Completed training epoch 8
Completed training epoch 9
Completed training epoch 10
Completed training epoch 11
Completed training epoch 12
Completed training epoch 13
Completed training epoch 14
Completed training epoch 15
Completed training epoch 16
Completed training epoch 17
Completed training epoch 18
Completed training epoch 19


In [16]:
# Now, let's save the model just in case
# ONLY RUN THIS ONCE
torch.save(model.state_dict(), './drive/MyDrive/CS 8395 - C-DiffNet/MNIST_Epochs/model')

In [9]:
model_test = MLP()

model_test.load_state_dict(torch.load('./drive/MyDrive/CS 8395 - C-DiffNet/MNIST_Epochs/model'))
model_test.eval()

classifier_params = torch.tensor(np.load('./drive/MyDrive/CS 8395 - C-DiffNet/MNIST_Epochs/new_dataset/epoch_10/model_0_params')[0])

model_test.classifier.weight = nn.Parameter(classifier_params[:32*10].reshape(10, 32))
model_test.classifier.bias = nn.Parameter(classifier_params[32*10:])

correct = 0
total = 0

for images, labels in test_loader:
  preds = model_test(images)
  real_vals = torch.argmax(preds, dim=-1)
  total += len(preds)
  cs = (real_vals == labels).sum().item()

  correct += cs
print(100 * correct / total)


97.44


In [21]:
# Now, lets freeze the params
for name, param in model_test.named_parameters():
  if name not in ['classifier.weight', 'classifier.bias']:
    param.requires_grad = False
  else:
    print(param.requires_grad)

import math
print(model_test.classifier.weight)
print(model_test.classifier.bias)
stdv = 1. / math.sqrt(model_test.classifier.weight.size(1))
model_test.classifier.weight.data.uniform_(-stdv, stdv)
if model_test.classifier.bias is not None:
    model_test.classifier.bias.data.uniform_(-stdv, stdv)
print('--------------------------------------------------')
print(model_test.classifier.weight)
print(model_test.classifier.bias)

True
True
Parameter containing:
tensor([[-3.6680e-02,  1.8201e-01,  9.7025e-02, -2.5817e-01, -2.0703e-01,
         -4.1172e-01,  6.7723e-02, -1.4132e-02,  6.7638e-02,  6.3527e-02,
          7.6617e-02, -2.3984e-01,  3.0244e-01, -1.4388e-01, -4.3175e-01,
         -3.8022e-01, -3.2880e-02,  2.8051e-01,  3.0463e-03, -2.2626e-01,
          8.7508e-02, -1.8252e-01, -1.3993e-01,  1.2301e-01,  1.1746e-01,
          7.3477e-02,  1.3556e-01, -8.3496e-02,  8.2460e-02, -7.8303e-01,
         -3.6823e-01, -1.6426e-01],
        [ 7.0923e-02,  1.0781e-01,  1.6201e-01,  1.8952e-01,  1.9553e-01,
         -2.5437e-01,  1.0791e-03, -1.0309e-01, -4.1695e-01,  3.6633e-01,
         -2.8174e-01, -8.6777e-02, -2.6997e-01, -8.1807e-02, -2.6683e-01,
          2.8912e-01,  2.1774e-01, -4.2366e-01, -3.7160e-01,  2.2341e-01,
          4.2955e-02, -1.6080e-01,  4.4884e-03,  1.4984e-02, -1.4511e-01,
         -3.5781e-01,  1.5950e-01, -2.5772e-02, -2.5241e-01, -1.5859e-01,
         -1.5014e-01,  2.6207e-01],
        

In [24]:
def create_base_models(num_models):
  models = []
  opts = []

  for _ in range(num_models):
    model = MLP()
    model.load_state_dict(torch.load('./drive/MyDrive/CS 8395 - C-DiffNet/MNIST_Epochs/model'))

    opt = torch.optim.Adam(model.parameters(), lr=1e-3)

    stdv = 1. / math.sqrt(model.classifier.weight.size(1))
    model.classifier.weight.data.uniform_(-stdv, stdv)
    if model.classifier.bias is not None:
        model.classifier.bias.data.uniform_(-stdv, stdv)

    model.classifier.weight.requires_grad = True
    model.classifier.bias.requires_grad = True

    for name, param in model.named_parameters():
      if name not in ['classifier.weight', 'classifier.bias']:
        param.requires_grad = False

    models.append(model)
    opts.append(opt)

  return models, opts

In [ ]:
from npy_append_array import NpyAppendArray
from pathlib import Path

loss_func = nn.CrossEntropyLoss()

new_models, new_opts = create_base_models(50)

for epoch in range(20):

  for model_num in range(len(new_models)):

    new_models[model_num].to(device)
    new_models[model_num].train()
    params = []
    for image, label in train_loader:
      image = image.to(device)
      label = label.to(device)
      new_opts[model_num].zero_grad()
      output = new_models[model_num](image.view(-1, 28*28))
      loss = loss_func(output, label)
      loss.backward()
      new_opts[model_num].step()

    new_models[model_num].to('cpu') # Bring to CPU to copy over
    params = new_models[model_num].classifier.parameters()
    params = nn.utils.parameters_to_vector(params).detach().numpy()
    # weights = models[model_num].classifier.weight.data
    # bias = models[model_num].classifier.bias.data

    dir = f'{DRIVE}new_dataset/epoch_{epoch}/'
    Path(dir).mkdir(parents=True, exist_ok=True)
    with NpyAppendArray(dir + f'model_{model_num}_params') as npaa:
      # Weights =  32 * 10
      # Bias = 10
      # First 32*10 are weights and last 10 are bias
      npaa.append(np.expand_dims(params,axis=0))

  print(f"Completed epoch {epoch} data")

Completed epoch 0 data
Completed epoch 1 data
Completed epoch 2 data
Completed epoch 3 data
Completed epoch 4 data
Completed epoch 5 data
Completed epoch 6 data
Completed epoch 7 data
Completed epoch 8 data
Completed epoch 9 data
